In [1]:
# PyTorch does not require project activation like Julia.
# Install packages if needed:
# !pip install torch torchvision

In [2]:
import torch
import json
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

with open('data/imdb_dataset_prepared.json', 'r') as f:
    data = json.load(f)

# Load your preprocessed data
# Assuming data is stored in npy format or similar for Python compatibility
X_train = torch.tensor(data['X_train'], dtype=torch.long)-1
y_train = torch.tensor(data['y_train'], dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)


X_test = torch.tensor(data['X_test'], dtype=torch.long)-1
y_test = torch.tensor(data['y_test'], dtype=torch.long)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64)

In [3]:
import torch.nn as nn

vocab_size = X_train.max().item()+1  # or use len(vocab)
embedding_dim = 50

class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, 100, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)  # batch x channels x seq_len
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.flatten(x)
        return self.fc(x)

num_classes = len(torch.unique(y_train))
model = CNNTextClassifier(vocab_size, embedding_dim, num_classes)

In [4]:
# Load pretrained GloVe embeddings (assumed to be preprocessed into a numpy array matching vocab size)
embedding_tensor = torch.tensor(data['embeddings'], dtype=torch.float)
model.embedding.weight.data.copy_(embedding_tensor)

tensor([[ 9.0951e-01, -2.0702e-01, -9.0611e-02,  ..., -7.7261e-01,
         -6.0396e-01, -3.1136e-01],
        [-5.8014e-01, -1.1316e+00,  4.4189e-01,  ..., -5.6616e-02,
          1.7220e-01,  1.2622e+00],
        [ 2.7137e-01,  6.1347e-01, -5.2498e-01,  ..., -1.7192e+00,
         -2.6018e-01,  5.7005e-01],
        ...,
        [ 4.5505e-01, -1.4904e-03, -4.5487e-01,  ..., -1.0215e+00,
          9.8155e-01, -5.0673e-01],
        [ 1.4323e-02, -7.4624e-01,  3.5701e-01,  ..., -4.6867e-01,
         -7.9873e-02, -8.9164e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

In [5]:
import time
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import psutil
import os

batch_size = 64
dataloader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

optimizer = Adam(model.parameters())
criterion = CrossEntropyLoss()

process = psutil.Process(os.getpid()) 

for epoch in range(5):
    total_loss = 0.0
    total_acc = 0.0
    num_batches = 0

    mem_before = process.memory_info().rss  # bytes

    start_time = time.time()
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

        predicted_labels = preds.argmax(dim=1)
        total_acc += (predicted_labels == yb).float().mean().item()

    elapsed = time.time() - start_time
    mem_after = process.memory_info().rss  # bytes

    avg_loss = total_loss / num_batches
    avg_acc = total_acc / num_batches * 100  # percentage

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            preds = model(xb)
            pred_labels = preds.argmax(dim=1)
            correct += (pred_labels == yb).sum().item()
            total += yb.size(0)

    test_acc = correct / total * 100

    mem_used_MB = (mem_after - mem_before) / (1024 ** 2)

    print(f"Epoch: {epoch+1} ({elapsed:.2f}s) \tTrain: (l: {avg_loss:.2f}, a: {avg_acc:.2f}%) "
          f"\tTest: (a: {test_acc:.2f}%) \tMemory Allocated: {mem_used_MB:.3f} MB")

Epoch: 1 (7.73s) 	Train: (l: 0.42, a: 80.40%) 	Test: (a: 85.02%) 	Memory Allocated: 20.293 MB
Epoch: 2 (8.88s) 	Train: (l: 0.28, a: 88.24%) 	Test: (a: 87.66%) 	Memory Allocated: -0.211 MB
Epoch: 3 (14.51s) 	Train: (l: 0.21, a: 91.92%) 	Test: (a: 88.11%) 	Memory Allocated: -0.926 MB
Epoch: 4 (14.01s) 	Train: (l: 0.14, a: 95.25%) 	Test: (a: 87.91%) 	Memory Allocated: -0.043 MB
Epoch: 5 (14.67s) 	Train: (l: 0.09, a: 97.58%) 	Test: (a: 87.74%) 	Memory Allocated: -0.043 MB
